In [1]:
from web3 import Web3
import pandas as pd
import datetime
from concurrent.futures import ProcessPoolExecutor
import numpy as np

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
#key_1 = "4d08ab7e29294be397e15ec5503401ca"
key_2 = "d4d8ddd18ef6438ea49fd3d5006b7eba"
key_3 = "b7b06ad6a7304e2197efa10b79e1c867"
key_4 = "58bce48fe80b451c96fee8ef35bbcce8"

provider = "https://mainnet.infura.io/v3/"

In [3]:
from web3 import Web3
import pandas as pd
from datetime import datetime
import numpy as np
from joblib import Parallel, delayed
import logging, os, json


logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

data_dir = 'block_data'
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

def save_block_data(block_number, data):
    """Save block transaction data to a JSON file."""
    try:
        for transaction in data:
            # Convert datetime objects to strings in ISO format before saving
            if 'timestamp' in transaction and isinstance(transaction['timestamp'], datetime):
                transaction['timestamp'] = transaction['timestamp'].isoformat()
            
            # Convert NumPy int64 to Python int
            for key, value in transaction.items():
                if isinstance(value, np.int64):
                    transaction[key] = int(value)
        
        file_path = os.path.join(data_dir, f'{block_number}.json')
        with open(file_path, 'w') as file:
            json.dump(data, file)
        logging.info(f"Block {block_number} data saved.")
    except Exception as e:
        logging.error(f"Error saving data for block {block_number}: {e}")


def load_block_data(block_number):
    """Check if block transaction data exists as a .pkl file."""
    file_path = os.path.join(data_dir, f'{block_number}.json')
    if os.path.exists(file_path):
        print(f'Block {block_number} data already exists')
        return True
    return False



w3 = Web3(Web3.HTTPProvider(provider + key_4))
start_block = 18935888
end_block   = 18993488


transactions_data = []
block_range = np.arange(start_block, end_block + 1)
filtered_block_range = [block for block in block_range if not load_block_data(block)]

print(len(filtered_block_range)/len(block_range)*100,"% of blocks are new")

for block_number in filtered_block_range:
    block_number_int = int(block_number)
    block = w3.eth.get_block(block_number_int, full_transactions=True)
    block_timestamp = datetime.utcfromtimestamp(block.timestamp)
    
    for tx in block.transactions:
        tx_receipt = w3.eth.get_transaction_receipt(tx.hash)
        
        transactions_data.append({
            'block_number': block_number,
            'tx_hash': tx.hash.hex(),
            'gas_price': tx.gasPrice,
            'gas_used': tx_receipt.gasUsed,
            'timestamp': block_timestamp
        })
        
    save_block_data(block_number, transactions_data)

print("Dataframe saved. Number of transactions:", len(transactions_data))

df_transactions = pd.DataFrame(transactions_data)

df_transactions['gas_price_gwei'] = df_transactions['gas_price'].apply(lambda x: w3.from_wei(x, 'gwei')).astype(int)
df_transactions['tx_hash']        = df_transactions['tx_hash'].astype(str)
df_transactions['timestamp']      = pd.to_datetime(df_transactions['timestamp'])
df_transactions = df_transactions.set_index('timestamp').sort_index()

df_transactions.to_pickle('ethereum_transactions.pkl')

Block 18935888 data already exists
Block 18935889 data already exists
Block 18935890 data already exists
Block 18935891 data already exists
Block 18935892 data already exists
Block 18935893 data already exists
Block 18935894 data already exists
Block 18935895 data already exists
Block 18935896 data already exists
Block 18935897 data already exists
Block 18935898 data already exists
Block 18935899 data already exists
Block 18935900 data already exists
Block 18935901 data already exists
Block 18935902 data already exists
Block 18935903 data already exists
Block 18935904 data already exists
Block 18935905 data already exists
Block 18935906 data already exists
Block 18935907 data already exists
Block 18935908 data already exists
Block 18935909 data already exists
Block 18935910 data already exists
Block 18935911 data already exists
Block 18935912 data already exists
Block 18935913 data already exists
Block 18935914 data already exists
Block 18935915 data already exists
Block 18935916 data 

2024-02-09 20:49:20,637 - INFO - Block 18936445 data saved.
2024-02-09 20:49:35,748 - INFO - Block 18936446 data saved.
2024-02-09 20:49:56,001 - INFO - Block 18936447 data saved.
2024-02-09 20:50:12,639 - INFO - Block 18936448 data saved.


In [12]:
filtered_block_range = [block for block in block_range if not load_block_data(block)]

In [17]:
load_block_data(18935888)

False

In [19]:
w3.eth.get_block(18935889, full_transactions=True)

AttributeDict({'baseFeePerGas': 23761180396,
 'difficulty': 0,
 'extraData': HexBytes('0x6265617665726275696c642e6f7267'),
 'gasLimit': 30000000,
 'gasUsed': 11550003,
 'hash': HexBytes('0xc1452e2e63e30e85b24b662016c0e5ef132e30afa2dacc896517863498ff0f49'),
 'logsBloom': HexBytes('0x00a7244d6850070950484d69822570b5a38841490427654ca28b086b82236dac901344981040a46001100388630405ad0f53510cbc9834b901ca749e442809711010805a41581ab8792a583cc122e0f8829a0050a3e43a2315959342a43d06847348584402ef831401a83a84400d2f5b2230194004050c5cd34020d01e18000abe03a740802ab9630e55a544a9d809172202628999116d9a2d17635e115143e8caa2cf1409886ce0665474d031c07d5a160f0f48ab3418c705214586018a10778b2670d654021e6b49598024614a8cc4002902ad2cd00a3e9e2a8a23c810a54a0135301fada42098572f2592cd1949d8a64edc42504e8045003b595c7ea6050b'),
 'miner': '0x95222290DD7278Aa3Ddd389Cc1E1d165CC4BAfe5',
 'mixHash': HexBytes('0xab50c2530d6f95ff05377c4717a4f163b81bf5dd71e6632126c60ae18d73401f'),
 'nonce': HexBytes('0x0000000000000000'),
 'number': 